In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [51]:
df=pd.read_csv('Churn_Modelling.csv')

In [52]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [53]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [54]:
label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df['Gender'])

In [55]:
one_hot_encoder = OneHotEncoder()
geo_encoded = one_hot_encoder.fit_transform(df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(['Geography']))

In [56]:
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [57]:
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

In [58]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [59]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [60]:
## Save the encoders and scaler
with open('label_encoder_gender1.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder1.pkl','wb') as file:
    pickle.dump(one_hot_encoder,file)
with open('scaler1.pkl','wb') as file:
    pickle.dump(scaler,file)

In [61]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [62]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

In [63]:
from keras.optimizers import Adam
from keras.losses import mean_absolute_error
adam=Adam()

In [64]:
model.compile(optimizer=adam,loss=mean_absolute_error,metrics=['mae'])

In [65]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                832       
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir = "regression/fit/"+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [67]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [68]:
history = model.fit(
    X_train,y_train,
    validation_data = (X_test,y_test),
    epochs=100,
    callbacks = [early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 100384.9062 - mae: 100384.9062 - val_loss: 98544.8203 - val_mae: 98544.8203
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99737.4531 - mae: 99737.4531 - val_loss: 97236.3516 - val_mae: 97236.3516
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 97465.3203 - mae: 97465.3203 - val_loss: 93899.5859 - val_mae: 93899.5859
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 92956.6406 - mae: 92956.6406 - val_loss: 88246.3828 - val_mae: 88246.3828
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 86223.8125 - mae: 86223.8125 - val_loss: 80766.9531 - val_mae: 80766.9531
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 78050.0625 - mae: 78050.0625 - val_loss: 72549.5781 - val_mae: 72549.5781
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 69683.5938 - mae: 69683.5938 

In [69]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
%tensorboard --logdir regression/fit/

Reusing TensorBoard on port 6006 (pid 15452), started 0:21:20 ago. (Use '!kill 15452' to kill it.)

In [71]:
test_loss,test_mae=model.evaluate(X_test,y_test)

63/63 [==============================] - 0s 807us/step - loss: 50322.2461 - mae: 50322.2461


In [72]:
model.save('regression_model.h5')

e:\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
